# ICILS dashboard IA (escalable)

Este es un proyecto personal que se enmarca en la organización SocialTec Lab. Es una exploración de datos de ICILS 2023 en el nuevo cuestionario de inteligencia artificial, utilizando datos de diversos países e implementando visualizaciones interactivas. Se utilizará panel y bokeh / holoviews y ploty para gráficos. Es un proyecto con intención de ser escalable para utilizarse para otras dashboards con ICILS. Utiliza pandas para la manipulación de datos. 

## Importamos los paquetes y datos.

In [3]:
import hvplot.pandas
import numpy as np
import pandas as pd
import panel as pn
import geopandas as gpd



### Procedimiento mapa

#### Leemos los datos para el mapa

In [6]:
# Load geographical data from parquet file
datos_mapa = pd.read_parquet('./output/datos_mapa.parquet', engine='pyarrow')


#### Creamos el widget para seleccionar la variable

In [8]:
import plotly.express as px
import panel as pn
import plotly.graph_objects as go

# Configuración para panel
pn.extension('plotly')

# Crear los widgets para cambiar la variable a visualizar
variable_select = pn.widgets.RadioButtonGroup(
    name='Variable a visualizar', 
    options=['IDI', 'CIL'],
    button_type='primary',
    value='IDI',
    button_style='solid',
    align="center"
)

#### Creamos la función para la visualización

In [10]:
# Crear una función para actualizar el mapa
def update_map(variable):
    updated_fig = px.choropleth(
        datos_mapa,
        locations='CNT',
        color=variable,
        hover_name='label',
        hover_data={
            'CNT': True,
            'CIL': ':.2f',
            'IDI': ':.2f'
        },
        projection='natural earth',
        title=f'{"Competencia Digital (CIL)" if variable == "CIL" else "Índice de Desarrollo TIC (IDI)"} por País',
        color_continuous_scale='sunsetdark'
    )
    
    # Usar las mismas configuraciones de diseño que en el original
    updated_fig.update_layout(
        margin={"r":10,"t":50,"l":10,"b":10},
        coloraxis_colorbar=dict(
            title='CIL' if variable == 'CIL' else 'IDI',
            thicknessmode="pixels", thickness=25,
            lenmode="pixels", len=400,
            ticks="outside",
            tickfont=dict(color='#e0e0e0', size=14),
            title_font=dict(color='#e0e0e0', size=16)
        ),
        geo=dict(
            showframe=False,
            showcoastlines=True,
            coastlinecolor='#4d5154',
            projection_type='natural earth',
            landcolor='#2e3135',
            showocean=True,
            oceancolor='#1c1f24',
            showlakes=False,
            lakecolor='#1c1f24',
            showrivers=False,
            rivercolor='#1c1f24',
            bgcolor='#121212',
            center=dict(lon=0, lat=20),
            projection_scale=1.0
        ),
        title=dict(
            text=f'{"Competencia Digital (CIL)" if variable == "CIL" else "Índice de Desarrollo TIC (IDI)"} por País',
            font=dict(size=28, color='#e0e0e0'),
            x=0.5,
            y=0.95
        ),
        paper_bgcolor='#121212',
        plot_bgcolor='#121212',
        font=dict(family="Arial, sans-serif", color='#e0e0e0', size=14)
    )
    
    # Mantener las mismas configuraciones de geo y hover
    updated_fig.update_geos(
        showcountries=True,
        countrycolor='#4d5154',
        countrywidth=0.8,
        showsubunits=True,
        subunitcolor='#4d5154',
        subunitwidth=0.8
    )
    
    updated_fig.update_traces(
        hovertemplate='<b>%{hovertext}</b><br>'+
                     (f'CIL Media: %{{z:.2f}}<br>IDI: %{{customdata[0]:.2f}}' if variable == 'CIL' else
                      f'IDI: %{{z:.2f}}<br>CIL Media: %{{customdata[0]:.2f}}') +
                     '<extra></extra>',
        customdata=datos_mapa[['IDI' if variable == 'CIL' else 'CIL']],
        marker_line_color='#4d5154',
        marker_line_width=0.8,
        hoverlabel=dict(
            bgcolor="#2e3135",
            font_size=16,
            font_family="Arial",
            font_color="#e0e0e0"
        )
    )
    
    return updated_fig

#### Hacemos el bind y los layouts para el mapa y su descripción

In [12]:
# Crear un panel interactivo
interactive_map = pn.bind(update_map, variable_select)

# Agregar una descripción con formato para modo oscuro
titulo_mapa = pn.pane.Markdown("""
## Mapa Global de Desarrollo de Tecnologías y Competencias Digitales
""", align="center", styles = {'text-align': 'center', 'color': '#c0c0c0', 'font-size': '16px'})

texto_mapa = pn.pane.Markdown("""
El siguiente mapa muestra la distribución global de alfabetización informática y computacional (CIL por sus siglas en inglés) y el Índice de Desarrollo de TIC (IDI) por país.
Los puntajes CIL se extraen de la prueba internacional ICILS 2023, que mide las habilidades para utilizar herramientas digitales en jóvenes de octavo grado.
""", styles={'text-align': 'justify', 'color': '#c0c0c0', 'font-size': '16px'})

texto_mapa_rw = pn.Row(pn.Spacer(width=50), texto_mapa, pn.Spacer(width=100))

mapa = pn.Row(
    pn.Spacer(width=100),
    pn.pane.Plotly(interactive_map, height=600, sizing_mode="scale_width"),
    pn.Spacer(width=100)
)

columna_mapa = pn.Column(
    titulo_mapa,
    texto_mapa_rw,
    pn.Spacer(height=50),
    variable_select,
    mapa
)

### Procedimiento para análisis detallado por país

#### Cargamos y procesamos los datos
nota: fueron previamente procesados en R.

In [15]:
import panel as pn
import pandas as pd

datos_biv = pd.read_parquet('./output/datos_biv.parquet', engine='pyarrow')

datos_biv['mig'] = datos_biv['mig'].map({0: 'No migrante', 1: 'Migrante'})

datos_biv['sex'] = datos_biv['sex'].map({0: 'Hombre', 1: 'Mujer'})

#### Creamos Widget para seleccionar país y título de sección

In [17]:
select_CNT = pn.widgets.Select(options= datos_biv['label'].unique().tolist(),
                            value="")

def get_CNT(label):
    return (
        f"## Exploración de CIL y variables contextuales: {label}"
    )

get_title = pn.bind(
    get_CNT, label = select_CNT
)

titulo_pais = pn.pane.Markdown(get_title, styles={'text-align': 'center', 'color': '#c0c0c0', 'font-size': '16px'})

titulo_seccion2 = pn.Row(titulo_pais, align = "center")

texto_seccion2 = pn.pane.Markdown("""
    A continuación, se encuentra un panel interactivo para analizar caso a caso la alfabetización informática e informacional en cada país. Debes comenzar escogiendo un país.
    Luego, se mostrará el promedio CIL y el IDI del país. Abajo de estos puntajes, se encuentra un gráfico interactivo, analizando relación entre CIL, estatus migratorio, género y alfabetización del hogar.
    """, styles={'text-align': 'justify', 'color': '#c0c0c0', 'font-size': '16px'})

texto_seccion2_rw = pn.Row(pn.Spacer(width=50), texto_seccion2, pn.Spacer(width=50))
                           

#### Creamos tarjetas para ver datos del país y mejoramos estilo

In [19]:
def get_mean(label):
    # Filtrar el DataFrame para la etiqueta indicada
    subset = datos_mapa[datos_mapa['label'] == label]
    if not subset.empty:
        return subset['CIL'].iloc[0]  # Retorna el primer valor encontrado
    else:
        return None  # O maneja el caso en que no se encuentre la etiqueta
        
get_mean_bd = pn.bind(
    get_mean, label = select_CNT
)

def get_IDI(label):
    # Filtrar el DataFrame para la etiqueta indicada
    subset = datos_mapa[datos_mapa['label'] == label]
    if not subset.empty:
        return subset['IDI'].iloc[0]  # Retorna el primer valor encontrado
    else:
        return None  # O maneja el caso en que no se encuentre la etiqueta

get_IDI_bd = pn.bind(
    get_IDI, label = select_CNT
)

CIL = pn.indicators.Number(name='Promedio Alfabetización Digital', value=get_mean_bd, format='{value:.2f}', default_color = "#ffb92c")

IDI = pn.indicators.Number(name='Indice de desarrollo de TICs', value=get_IDI_bd, format='{value:.2f}', default_color = "#ff1a93")

titulo_widget = pn.pane.Markdown("#### Escoge un país", styles={'color': '#e0e0e0'}, align="start")

widget_cd = pn.Card(
    pn.Spacer(height=20),
    titulo_widget,
    pn.Row(select_CNT, sizing_mode="stretch_width"),
    pn.Spacer(height=20),
    max_width=333,
    collapsible=False,
    hide_header=True,
    styles={'background': '#000'}
)

CIL_cd = pn.Card(
    pn.Row(CIL, sizing_mode="stretch_width"),
    max_width=333,
    collapsible=False,
    hide_header=True,
    styles={'background': '#000'}
)

IDI_cd = pn.Card(
    pn.Row(IDI, sizing_mode="stretch_width"),
    max_width=333,
    height=181,
    collapsible=False,
    hide_header=True,
    styles={'background': '#000'}
)

puntajes_pais_rw = pn.Row(
    pn.Spacer(width=250),
    widget_cd,
    pn.Spacer(width=50),
    CIL_cd, 
    pn.Spacer(width=50),
    IDI_cd, 
    sizing_mode = "stretch_width",
    align="center"
)



#### Creamos gráfico de barras asociado a un widget que escoge la X

In [21]:
#pn.pane.Markdown(f"## Puntaje CIL y características contextuales: {label}")
import pandas as pd
import plotly.express as px

pn.extension("plotly")

select_variable_dic = pn.widgets.RadioButtonGroup(
    options={'Género': 'sex', 'Estatus migrante': 'mig', 'Alfabetización hogar': 'home_literacy'},
    button_type='primary',
    value='sex',  # Debe coincidir con un valor del diccionario
    button_style='solid',
    align="center"
)

def get_plot(label, variable):
    subset = datos_biv[datos_biv['label'] == label]
    df_promedio = subset.groupby(variable, as_index=False).agg({
    'CIL': 'mean',
    'home_literacy': 'mean',
    'i_SEB': 'mean'
    }).round(2)
    min_val = df_promedio['CIL'].min()
    max_val = df_promedio['CIL'].max()
    delta = max_val - min_val
    expanded_range = [min_val - delta, max_val + delta]
    fig = px.bar(df_promedio, x=variable, y='CIL', color='CIL',
             title=f"Diferencia en promedio CIL: {variable} en {label}",
             range_color=expanded_range,
             color_continuous_scale='sunsetdark',
             hover_data=['home_literacy', 'i_SEB'],
             labels={'CIL':'Promedio CIL'}, height=400)
    fig.update_layout(
    plot_bgcolor="#000000",  
    paper_bgcolor="#000000",
    font=dict(color="white"),  # Cambia el color de todos los textos en el gráfico
    hoverlabel=dict(
        bgcolor="black",   # Fondo negro para los tooltips
        font=dict(color="white")  # Texto blanco en los tooltips
    )
    )
    return fig

plot_mig_bd = pn.bind(get_plot, label=select_CNT, variable = select_variable_dic)

plot_mig_sex = pn.pane.Plotly(plot_mig_bd, sizing_mode="fixed", width=800, height=500, align="center")

plot_mig_sex_cn = pn.Column(select_variable_dic, plot_mig_sex, align="center")


#### Creamos la columna 

In [23]:
columna_pais = pn.Column(
    titulo_seccion2,
    texto_seccion2_rw,
    pn.Spacer(height=50),
    puntajes_pais_rw,
    pn.Spacer(height=50),
    plot_mig_sex_cn
)

#### Agregamos CSS personalizado para la dashboard

In [25]:
# Usar un CSS mínimo para centrar los botones
css = """
:root {
    --bs-body-bg: #02202d !important; /* Nuevo color de fondo */
}

.navbar-expand-md {
    background-color: black !important;
    transition: background-color 0.3s ease-in-out !important; /* Transición suave */
}

/* Navbar con efecto al hacer scroll */
.navbar-expand-md.scrolled {
    background-color: rgba(75, 0, 130, 0.6) !important; /* Morado oscuro semi-transparente */
}
.bk-btn-group {
    display: flex;
    justify-content: center;
    width: 100%;
}

.bg-primary {
    background-color: #02202d !important;
    color: white; /* Asegura contraste */
}

.bk-btn.bk-btn-primary {
    background-color:#ff1a93 !important;
}

"""


### Creamos el layout

In [80]:
pn.extension(raw_css=[css])
# pn.config.template = 'bootstrap'
# pn.template.BootstrapTemplate(title="CIL and IDI Dashboard")

sitio = pn.Column(
    columna_mapa,
    pn.Spacer(height=100),
    columna_pais
)

template = pn.template.BootstrapTemplate(
    title='Desarrollo y capacidades tecnológicas alrededor del mundo',
    main=sitio
)

template.servable()

BootstrapTemplate
    [js_area] HTML(None, height=0, margin=0, sizing_mode='fixed', width=0)
    [actions] BootstrapTemplateActions()
    [browser_info] BrowserInfo(dark_mode=True, device_pixel_ratio=1.1320754716981132, language='es-ES', timezone='America/Santiago', timezone_offset=180, webdriver=False, webgl=True)
    [busy_indicator] LoadingSpinner(height=20, width=20)
    [main-1804339092064] Column
        [0] Markdown(str, align='center', styles={'text-align': 'center', ...})
        [1] Row
            [0] Spacer(width=50)
            [1] Markdown(str, styles={'text-align': 'justify', ...})
            [2] Spacer(width=100)
        [2] Spacer(height=50)
        [3] RadioButtonGroup(align='center', button_type='primary', name='Variable a visualizar', options=['IDI', 'CIL'], value='IDI')
        [4] Row
            [0] Spacer(width=100)
            [1] Plotly(Figure, _render_count=5, height=600, sizing_mode='scale_width')
            [2] Spacer(width=100)
    [main-1804408786144] Spacer(height=100)
    [main-1804360141664] Column
        [0] Row(align='center')
            [0] Markdown(str, styles={'text-align': 'center', ...})
        [1] Row
            [0] Spacer(width=50)
            [1] Markdown(str, styles={'text-align': 'justify', ...})
            [2] Spacer(width=50)
        [2] Spacer(height=50)
        [3] Row(align='center', sizing_mode='stretch_width')
            [0] Spacer(width=250)
            [1] Card(collapsible=False, hide_header=True, max_width=333, styles={'background': '#000'})
                [0] Spacer(height=20)
                [1] Markdown(str, styles={'color': '#e0e0e0'})
                [2] Row(sizing_mode='stretch_width')
                    [0] Select(options=['Austria', 'Azerbaijan', ...], value='Austria')
                [3] Spacer(height=20)
            [2] Spacer(width=50)
            [3] Card(collapsible=False, hide_header=True, max_width=333, styles={'background': '#000'})
                [0] Row(sizing_mode='stretch_width')
                    [0] Number(default_color='#ffb92c', format='{value:.2f}', name='Promedio Alfabetización D..., value=514.8960709587019)
            [4] Spacer(width=50)
            [5] Card(collapsible=False, height=181, hide_header=True, max_width=333, styles={'background': '#000'})
                [0] Row(sizing_mode='stretch_width')
                    [0] Number(default_color='#ff1a93', format='{value:.2f}', name='Indice de desarrollo d..., value=92.5)
        [4] Spacer(height=50)
        [5] Column(align='center')
            [0] RadioButtonGroup(align='center', button_type='primary', options={'Género': 'sex', ...}, value='mig')
            [1] Plotly(Figure, _render_count=514, align='center', height=500, sizing_mode='fixed', width=800)

In [ ]:
!panel convert probe_of_concept.ipynb --to html --output mi_panel.html